In [24]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm, zscore
from scipy.optimize import brentq
from scipy.interpolate import SmoothBivariateSpline

#Black-Scholes model and implied volatility calculation via optimization
def black_scholes_call(S, K, T, r, sigma):
    if T <= 1e-6: return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def implied_volatility(price, S, K, T, r):
    if price <= max(S - K * np.exp(-r * T), 0) or price >= S: return np.nan
    try:
        return brentq(lambda sig: black_scholes_call(S, K, T, r, sig) - price, 1e-6, 4.0)
    except: return np.nan

#Data filtered
ticker_symbol = "GOOGL" 
ticker = yf.Ticker(ticker_symbol)
S_spot = ticker.history(period="1d")['Close'].iloc[-1]
r_rate = 0.035 

expirations = ticker.options[1:12]
raw_data = []

for exp in expirations:
    chain = ticker.option_chain(exp)
    calls = chain.calls[(chain.calls['strike'] > S_spot * 0.9) & (chain.calls['strike'] < S_spot * 1.1)]
    T_years = (pd.to_datetime(exp) - pd.Timestamp.now()).days / 365.0
    if T_years <= 0.05: continue
    for _, row in calls.iterrows():
        iv = implied_volatility(row['lastPrice'], S_spot, row['strike'], T_years, r_rate)
        if not np.isnan(iv):
            raw_data.append([T_years, row['strike'], iv])

df = pd.DataFrame(raw_data, columns=['T', 'K', 'IV'])

#Filter spiky noise
# We remove data points where the IV is statistically impossible compared to neighbors
df = df[np.abs(zscore(df['IV'])) < 2.0]
pts = df[['T', 'K']].values
vars_list = (df['IV']**2 * df['T']).values

#Smooth it even more
interp_vol_surf = SmoothBivariateSpline(pts[:, 0], pts[:, 1], vars_list, s=8.0)

#Dupire formula
def get_clean_vol(K, T):
    dk, dt = K * 0.01, 0.001
    def p(strike, time):
        v = max(interp_vol_surf(time, strike)[0][0], 1e-6)
        return black_scholes_call(S_spot, strike, time, r_rate, np.sqrt(v/time))
    
    C = p(K, T)
    dCdT = (p(K, T + dt) - C) / dt
    dCdK = (p(K + dk, T) - p(K - dk, T)) / (2 * dk)
    d2CdK2 = (p(K + dk, T) - 2*C + p(K - dk, T)) / (dk**2)
    
    num, den = dCdT + r_rate * K * dCdK, 0.5 * K**2 * d2CdK2
    if den <= 1e-7 or num <= 0: return np.nan
    return np.sqrt(num / den)

#Plotting
T_grid = np.linspace(df['T'].min(), df['T'].max() - 0.01, 40)
K_grid = np.linspace(df['K'].min(), df['K'].max(), 40)
T_mesh, K_mesh = np.meshgrid(T_grid, K_grid)
Z_loc = np.vectorize(get_clean_vol)(K_mesh, T_mesh)

# Smooth out artifacts
Z_loc = np.clip(np.nan_to_num(Z_loc, nan=np.nanmean(Z_loc)), 0.1, 0.6)

fig = go.Figure(data=[go.Surface(
    z=Z_loc, 
    x=T_grid, # Time on X
    y=K_grid, # Strike on Y
    colorscale='Viridis',
    hovertemplate='Maturity (T): %{x:.2f}<br>Strike (K): %{y:.2f}<br>Local Vol (σ): %{z:.2%}<extra></extra>'
)])

fig.update_layout(
    title=f"Dupire Local Volatility Surface: {ticker_symbol}",
    template="plotly_dark",
    scene=dict(
        xaxis_title='Maturity (T)',
        yaxis_title='Strike (K)',
        zaxis_title='Local Vol (σ)',
        # Optional: Adjust the aspect ratio to make it look less "squashed"
        aspectratio=dict(x=1, y=1, z=0.7)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()